# Tarea 1

## Recuperación ranqueada y vectorización de documentos (RRDV) usando GENSIM

In [10]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import zipfile
from gensim import corpora, models, similarities
from metrics import precision_at_k, recall_at_k, ndcg_at_k, mean_average_precision


In [11]:
# Se extraen los datos de los archivos comprimidos

# Specify the paths to the compressed files and the target directory
compressed_files = ['docs-raw-texts.zip', 'queries-raw-texts.zip']

num_documents = 0
# Extract files from each compressed file
for compressed_file in compressed_files:
    with zipfile.ZipFile(compressed_file, 'r') as zip_ref:
        folder_name = os.path.splitext(compressed_file)[0]  # Remove the ".zip" extension
        target_folder = os.path.join(folder_name)
        
        if not os.path.exists(target_folder):
            # Create the folder within the target directory
            os.mkdir(target_folder)

        
            # Extract all files to the target folder
            zip_ref.extractall(target_folder)

print("Extracción completada")

Extracción completada


In [12]:
# Extracts raw text from .naf files
def extract_raw_text(xml_path, title=False):
        if title:
            # Parse the XML file
            tree = ET.parse(xml_path)
            root = tree.getroot()
            # Extract content from the XML
            return root.find(".//nafHeader/fileDesc").get("title") + ", " + root.find('raw').text #Añade título cuando se especifíca
        else:
            # Parse the XML file
            tree = ET.parse(xml_path)
            root = tree.getroot()
            # Extract content from the XML
            return root.find('raw').text

# filtra los resultados de las listas para solo incluir los nombres de los documentos
def filter_result_list(input_list):
    return[full_result[:full_result.find(":")] for full_result in input_list]

In [13]:
# Load the relevance judgments from the file
relevance_judgments_file = open("relevance-judgments.tsv", "r")
relevance_judgments = {}

for line in relevance_judgments_file:
    query, judgments = line.strip().split('\t')
    relevance_judgments[query] = judgments.split(',')

relevance_judgments_file.close()

In [15]:
relevance_judgments


{'q01': ['d186:4', 'd254:5', 'd016:5'],
 'q02': ['d136:2',
  'd139:2',
  'd143:4',
  'd283:4',
  'd228:4',
  'd164:4',
  'd318:2',
  'd291:4',
  'd293:4',
  'd147:2',
  'd149:2'],
 'q03': ['d152:3', 'd291:4', 'd283:4', 'd147:3', 'd318:2', 'd105:2'],
 'q04': ['d275:3', 'd010:3', 'd286:2', 'd019:2', 'd049:2', 'd330:2', 'd270:3'],
 'q06': ['d069:2', 'd233:3', 'd257:2', 'd297:3', 'd026:4', 'd329:5'],
 'q07': ['d004:3', 'd077:3', 'd266:2', 'd179:3'],
 'q08': ['d205:2',
  'd005:4',
  'd110:4',
  'd108:3',
  'd117:3',
  'd081:2',
  'd292:2',
  'd251:5',
  'd028:3',
  'd271:3',
  'd121:2',
  'd180:2'],
 'q09': ['d205:3', 'd199:5', 'd198:3', 'd223:2', 'd217:2', 'd177:2'],
 'q10': ['d068:2',
  'd100:2',
  'd065:3',
  'd076:3',
  'd231:4',
  'd199:4',
  'd052:2',
  'd215:2'],
 'q12': ['d239:4', 'd277:4', 'd258:3', 'd250:4'],
 'q13': ['d239:2', 'd277:2', 'd258:2', 'd049:4', 'd056:4'],
 'q14': ['d002:2',
  'd005:3',
  'd142:2',
  'd314:3',
  'd280:3',
  'd130:3',
  'd041:3',
  'd117:2',
  'd081:4',